## Search Algorithms - A*

### Resources
#### A* search algorithm
- [Introduction to the A* Algorithm](https://www.redblobgames.com/pathfinding/a-star/introduction.html)
- [Implementation of A*](https://www.redblobgames.com/pathfinding/a-star/implementation.html#python)
- [Introduction to A*](https://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html)
- [Amit's A\* Pages - pathfinding](http://theory.stanford.edu/~amitp/GameProgramming/)
---
- [Grids and Graphs](https://www.redblobgames.com/pathfinding/grids/graphs.html)
- [How do we represent a grid’s obstacles in a graph form?](https://www.redblobgames.com/pathfinding/grids/graphs.html#obstacles)
- [Graph (abstract data type) - Wikipedia](https://en.wikipedia.org/wiki/Graph_(abstract_data_type)#Representations)
---
- [A* Search](https://www.codecademy.com/resources/docs/ai/search-algorithms/a-star-search)
- [A* Search Algorithm - stackabuse](https://stackabuse.com/courses/graphs-in-python-theory-and-implementation/lessons/a-star-search-algorithm/)
- [A* search algorithm - isaaccomputerscience](https://www.isaaccomputerscience.org/concepts/dsa_search_a_star)
---
- [Admissible heuristic](https://en.wikipedia.org/wiki/Admissible_heuristic)
#### Other Lectures
- [Breadth First Search](https://www.redblobgames.com/pathfinding/a-star/introduction.html#breadth-first-search)
- [Breadth first search: early exits](https://www.redblobgames.com/pathfinding/early-exit/)

`source`: https://www.redblobgames.com/pathfinding/a-star/introduction.html#more

> **Which algorithm should you use for finding paths on a game map?**
> 
> * If you want to find paths from or to all all locations, use Breadth First Search or Dijkstra’s Algorithm. Use Breadth First Search if movement costs are all the same; use Dijkstra’s Algorithm if movement costs vary.
> * If you want to find paths to one location, or the closest of several goals, use Greedy Best First Search or A\*. Prefer A\* in most cases. When you’re tempted to use Greedy Best First Search, consider using A\* with an “inadmissible” heuristic.

> **What about optimal paths?**
>
>  Breadth First Search and Dijkstra’s Algorithm are guaranteed to find the shortest path given the input graph. Greedy Best First Search is not. A\* is guaranteed to find the shortest path if the heuristic is never larger than the true distance. As the *heuristic becomes smaller*, **A\* turns into Dijkstra’s Algorithm**. As the *heuristic becomes larger*, **A\* turns into Greedy Best First Search**.

In [7]:
# source: https://www.redblobgames.com/pathfinding/a-star/implementation.html#python
#
# This script contains:
# - data structures: queue, priority queue, graph
# - implementation of search algorithms: a_star_search, dijkstra, breadth_first_search
# - utility function: reconstruct_path, draw_grid, ...
# - examples: example_graph, DIAGRAM1_WALLS, diagram4
import implementation

### EXAMPLES
Examples to show how to use functionalities on **implementation.py** script.


#### Resources
- [pathfinding - python implementation](https://www.redblobgames.com/pathfinding/a-star/implementation.html)

#### Directed Graph

![directed-graph](./images/directed_graph.png)

```python
class SimpleGraph:
    def __init__(self):
        self.edges: dict[Location, list[Location]] = {}
    
    def neighbors(self, id: Location) -> list[Location]:
        return self.edges[id]
```

In [8]:
from implementation import SimpleGraph

example_graph = SimpleGraph()
example_graph.edges = {
    'A': ['B'],
    'B': ['C'],
    'C': ['B', 'D', 'F'],
    'D': ['C', 'E'],
    'E': ['F'],
    'F': [],
}

print(f"graph edges: {example_graph.edges}")
print(f"C neighbors: {example_graph.neighbors('C')}")

graph edges: {'A': ['B'], 'B': ['C'], 'C': ['B', 'D', 'F'], 'D': ['C', 'E'], 'E': ['F'], 'F': []}
C neighbors: ['B', 'D', 'F']


#### Grid Graph

![grid-graph](./images/grid_graph.png)

```python
GridLocation = Tuple[int, int]

class SquareGrid(Graph):
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
    
    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls
    
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results
```

In [9]:
from implementation import SquareGrid, DIAGRAM1_WALLS

grid_graph = SquareGrid(30, 15)
print(f"DIAGRAM1_WALLS: {sorted(DIAGRAM1_WALLS, key=lambda t: (t[0],t[1]))}")
grid_graph.walls = DIAGRAM1_WALLS # long list, [(21, 0), (21, 2), ...]

DIAGRAM1_WALLS: [(3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (13, 4), (13, 5), (13, 6), (13, 7), (13, 8), (13, 9), (13, 10), (13, 11), (13, 12), (13, 13), (13, 14), (14, 4), (14, 5), (14, 6), (14, 7), (14, 8), (14, 9), (14, 10), (14, 11), (14, 12), (14, 13), (14, 14), (21, 0), (21, 1), (21, 2), (21, 3), (21, 4), (21, 5), (21, 6), (22, 0), (22, 1), (22, 2), (22, 3), (22, 4), (22, 5), (22, 6), (23, 5), (23, 6), (24, 5), (24, 6), (25, 5), (25, 6)]


In [10]:
from implementation import draw_grid

draw_grid(grid_graph)

__________________________________________________________________________________________
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

In [11]:
grid_location = (2, 3)
list(grid_graph.neighbors(grid_location))

[(1, 3), (2, 2), (2, 4)]

#### Graph with weights

```python
# abstract class to represent a weighte graph
class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location) -> float: pass

# WeightedGraph subclass for Grids with weigths that depend only on 'to_node' node
class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}
    
    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        return self.weights.get(to_node, 1)
```

In [12]:
from implementation import GridWithWeights

weighted_grid_graph = GridWithWeights(10, 10)
weighted_grid_graph.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
weighted_grid_graph.weights = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}


In [13]:
draw_grid(weighted_grid_graph)

______________________________
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#### A\* implementation

In [14]:
# source code extraced from 'implementation.a_star_search' function
from implementation import WeightedGraph, Location, PriorityQueue, GridLocation

# defining my heuristic (using manhattan distance)
def heuristic(a: GridLocation, b: GridLocation) -> float:
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

def a_star_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[Location, Optional[Location]] = {}
    cost_so_far: dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: Location = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

In [15]:
start, goal = (1, 4), (8, 3)

# searching the shortest path from start to goal using A* algorithm
came_from, cost_so_far = a_star_search(weighted_grid_graph, start, goal)

In [16]:
# drawing the graph
draw_grid(weighted_grid_graph, point_to=came_from, start=start, goal=goal)

______________________________
 v  v  v  v  <  <  <  <  <  < 
 v  v  v  v  <  ^  ^  <  <  < 
 v  v  v  v  <  <  ^  ^  <  < 
 >  v  <  <  <  <  .  ^  Z  . 
 >  A  <  <  <  .  .  .  .  . 
 ^  ^  ^  <  <  .  .  .  .  . 
 ^  ^  ^  <  <  .  .  .  .  . 
 ^ ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [17]:
from implementation import reconstruct_path

# drawing the found path (reconstructing the path)
draw_grid(weighted_grid_graph, path=reconstruct_path(came_from, start=start, goal=goal))

______________________________
 .  .  .  @  @  @  @  .  .  . 
 .  .  .  @  .  .  @  @  .  . 
 .  .  .  @  .  .  .  @  @  . 
 .  @  @  @  .  .  .  .  @  . 
 .  @  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [18]:
draw_grid(weighted_grid_graph, number=cost_so_far, start=start)

______________________________
 5  4  5  6  7  8  9  10 11 12
 4  3  4  5  10 13 10 11 12 13
 3  2  3  4  9  14 15 12 13 14
 2  1  2  3  8  13 .  17 14 . 
 1  A  1  6  11 .  .  .  .  . 
 2  1  2  7  12 .  .  .  .  . 
 3  2  3  4  9  .  .  .  .  . 
 4 ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Tarea 2 - A\*

### Enunciado
![tarea2-enunciado](./images/enunciado.jpeg)

### Graph
![tarea2-graph](./images/graph.jpeg)

---
![tarea2-graph-with-numeration](./images/graph_with_numeration.png)

![tarea2 - path](./images/graph_path.png)

---
##### Credits
- Joel Wilson

In [19]:
# In this case the graph hasn't weight (So we are going to consider that all the nodes have an uniform weight of 1)
#
# NOTE: 
# - For A* and dijkstra algorithms, we must pass as input an weighted graph

from implementation import GridWithWeights, draw_grid

#graph = GridWithWeights()
graph = GridWithWeights(6, 6)

# '###' represent a block
graph.walls = [
    # (x, y): starting from left upper corner
    (0, 3), (1, 3), (4, 3),
    (2, 1), (2, 2)
]

draw_grid(graph)

__________________
 .  .  .  .  .  . 
 .  . ### .  .  . 
 .  . ### .  .  . 
###### .  . ### . 
 .  .  .  .  .  . 
 .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~


In [20]:
from implementation import GridLocation
from math import sqrt

# defining my heuristic (using manhattan distance)
def heuristic(a: GridLocation, b: GridLocation) -> float:
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

In [21]:
from implementation import a_star_search

start, goal = (0, 5), (0, 1)

# searching the shortest path from start to goal using A* algorithm
came_from, cost_so_far = a_star_search(graph, start, goal)

In [22]:
from implementation import reconstruct_path

# drawing the found path (reconstructing the path)
draw_grid(graph, path=reconstruct_path(came_from, start=start, goal=goal))

__________________
 @  @  @  @  .  . 
 @  . ### @  .  . 
 .  . ### @  .  . 
###### @  @ ### . 
 @  @  @  .  .  . 
 @  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~


In [23]:
draw_grid(graph, number=cost_so_far, start=start)

__________________
 11 10 9  8  9  . 
 12 11### 7  8  . 
 .  . ### 6  7  . 
###### 4  5 ### . 
 1  2  3  4  5  . 
 A  1  2  3  4  . 
~~~~~~~~~~~~~~~~~~


### What is happening (WE ARE NOT MOVING IN DIAGONALS!!)

Checking the source code of **implementation** scripts. We notice that the method `SquareGrid.neighbors` only return neighbors on `North, West, South and East` directions. But in our case we can do diagonal movements.

```python
class SquareGrid:
    ...
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

```

Also the class `GridWithWeights` dont't consider the cost of diagonal movements.

```python
class GridWithWeights(SquareGrid):
    ...
    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        return self.weights.get(to_node, 1)
```

Modifing them:

* **SquareGrid** class

```python
class MySquareGrid(SquareGrid):
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [
            (x+1, y), (x-1, y), (x, y-1), (x, y+1),  # E W S N
            (x+1, y+1), (x+1, y-1), (x-1, y+1), (x-1, y-1), # diagonal movements
        ]
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results
```

* **WeightedGraph** class
```python
from math import sqrt

class MyWeightedGraph(MySquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}

    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        x0, y0 = from_node
        x1, y1 = to_node
        return sqrt((x1 - x0)**2 + (y1 - y0)**2)
```

In [24]:
from implementation import SquareGrid, GridLocation
from typing import Protocol, Iterator, Tuple, TypeVar, Optional

In [25]:
from math import sqrt

class MySquareGrid(SquareGrid):
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [
            (x+1, y), (x-1, y), (x, y-1), (x, y+1),  # E W S N
            (x+1, y+1), (x+1, y-1), (x-1, y-1), (x-1, y+1), # diagonal movements
        ]
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

class MyGridWithWeights(MySquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}

    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        x0, y0 = from_node
        x1, y1 = to_node
        return sqrt((x1 - x0)**2 + (y1 - y0)**2)


In [26]:
graph = MyGridWithWeights(6, 6)

# '###' represent a block
graph.walls = [
    # (x, y): starting from left upper corner
    (0, 3), (1, 3), (4, 3),
    (2, 1), (2, 2)
]

draw_grid(graph)

__________________
 .  .  .  .  .  . 
 .  . ### .  .  . 
 .  . ### .  .  . 
###### .  . ### . 
 .  .  .  .  .  . 
 .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~


In [27]:
from implementation import a_star_search

start, goal = (0, 5), (0, 1)

# searching the shortest path from start to goal using A* algorithm
came_from, cost_so_far = a_star_search(graph, start, goal)

In [28]:
from implementation import reconstruct_path

# drawing the found path (reconstructing the path)
draw_grid(graph, path=reconstruct_path(came_from, start=start, goal=goal))

__________________
 .  .  .  .  .  . 
 @  . ### .  .  . 
 .  @ ### .  .  . 
###### @  . ### . 
 .  @  .  .  .  . 
 @  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~


In [73]:
draw_grid(graph, number=cost_so_far, start=start)

__________________
 .  .  .  .  .  . 
 5  5 ### .  .  . 
 5  4 ### 4  .  . 
###### 2  3 ### . 
 1  1  2  4  .  . 
 A  1  2  .  .  . 
~~~~~~~~~~~~~~~~~~


#### UPS! Our algorithm is jumping walls

Let's restict the diagonal movements.

In [29]:
class MySquareGridV2(SquareGrid):
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [
            (x+1, y), (x-1, y), (x, y-1), (x, y+1),  # E W S N
            #(x+1, y+1), (x+1, y-1), (x+1, y-1), (x-1, y-1), # diagonal movements
        ]

        if not ((x-1, y) in self.walls and (x, y+1) in self.walls):
            neighbors.append((x-1, y+1)) # NW

        if not ((x-1, y) in self.walls and (x, y-1) in self.walls):
            neighbors.append((x-1, y-1)) # SW
            
        if not ((x, y-1) in self.walls and (x+1, y) in self.walls):
            neighbors.append((x+1, y-1)) # SE
            
        if not ((x, y+1) in self.walls and (x+1, y) in self.walls):
            neighbors.append((x+1, y+1)) # NE
            
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

class MyGridWithWeightsV2(MySquareGridV2):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}

    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        x0, y0 = from_node
        x1, y1 = to_node
        return sqrt((x1 - x0)**2 + (y1 - y0)**2)

In [30]:
graph = MyGridWithWeightsV2(6, 6)

# '###' represent a block
graph.walls = [
    # (x, y): starting from left upper corner
    (0, 3), (1, 3), (4, 3),
    (2, 1), (2, 2)
]

draw_grid(graph)

__________________
 .  .  .  .  .  . 
 .  . ### .  .  . 
 .  . ### .  .  . 
###### .  . ### . 
 .  .  .  .  .  . 
 .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~


In [31]:
from implementation import a_star_search

start, goal = (0, 5), (0, 1)

# searching the shortest path from start to goal using A* algorithm
came_from, cost_so_far = a_star_search(graph, start, goal)

In [32]:
from implementation import reconstruct_path

# drawing the found path (reconstructing the path)
draw_grid(graph, path=reconstruct_path(came_from, start=start, goal=goal))

__________________
 .  .  @  .  .  . 
 @  @ ### @  .  . 
 .  . ### @  .  . 
###### @  . ### . 
 .  @  .  .  .  . 
 @  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~


In [33]:
draw_grid(graph, number=cost_so_far, start=start)

__________________
 9  7  6  6  6  . 
 9  8 ### 5  5  . 
 9  9 ### 4  5  . 
###### 2  3 ### . 
 1  1  2  3  4  . 
 A  1  2  3  4  . 
~~~~~~~~~~~~~~~~~~
